###  Extracting The Busniss's Name, Adress, Phon Number & Website Of Each Business.

> ### Inputs Cell.

In [1]:
type = ['Buyers-Agent', 'Listing-Agent', 'Relocation', 'Foreclosure', 'Shor-tSale', 'Consulting', 'Other']
search_location = 'houston-tx'# make a check that search makes the 
agent_type = type[-2].lower()
no_of_results = 1000

> ### Dependecies Cell.

In [2]:
import re
from bs4 import BeautifulSoup
from lxml import etree
import requests
import lxml.html
import lxml.html.soupparser
from lxml.etree import tostring
from fake_useragent import UserAgent

import time
from time import sleep
import pandas as pd
import numpy as np

> ### Primary Scraper. 

In [3]:
print(agent_type, ' agents' + ' in ' + search_location)
start_time_0 = time.time()
result_list =[]
page = 0
while no_of_results > len(result_list):
    
    url =f"https://www.zillow.com/professionals/{agent_type}--real-estate-agent-reviews/{search_location}/"   
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
               "referer": url}    
     
    page += 1
    if page > 1:
        url = url+f'/?page={page}'
    
    """ scraping the internet"""      
    webpage = requests.get(url = url, headers=headers)
    path_soup = BeautifulSoup(webpage.content,"html.parser")
    path = etree.HTML(str(path_soup))
    soup = BeautifulSoup(webpage.text,"html.parser")

    """ working on the saved zillow html file"""
    # with open('zillow.html', 'r') as f:
    #     global webpage
    #     webpage = f.read()
    # soup = BeautifulSoup(webpage,'lxml')
    # path = etree.HTML(str(soup))


    no_of_pages_available = (path.xpath('//*[@id="__next"]/div/main/div/div/div[1]/section/nav/ul/li[*]'))[-2]
    pages_available = int((no_of_pages_available.xpath('./span/text()[4]'))[0].strip())

    primary_info  = (path.xpath('//*[@id="__next"]/div/main/div/div/div[1]/section/table/tbody/tr[*]'))
    sleep(2)
    
    for P in primary_info: 
        if no_of_results == len(result_list):
            break
        agent_E = (P.xpath('./td[1]/div/div[2]/a[1]'))[0]
        
        agent = agent_E.text
        
        employer = (P.xpath("./td[1]/div/div[2]/div[2]"))[0].text
        
        profile = "https://www.zillow.com/"+(agent_E.xpath('./@href'))[0]
        
        phone = (P.xpath('./td[1]/div/div[2]/div[1]/text()'))[0]

        ratting = (P.xpath('./td[1]/div/div[2]/span/svg[*]/title'))
        stars = 0
        for R in ratting:
            if R.text == 'Filled Star':
                stars += 1
            elif R.text == 'Half Filled Star':
                stars += 0.5
            else:
                stars += 0 
          
        result = {"Agent Name": agent, "Phone Number" :phone, "Profile": profile,
                "Ratting": stars, "Employer": employer } 
        result_list.append(result)
              
              
    if page == pages_available or page == (no_of_results/10):
        break
    
           
main_df = pd.DataFrame(result_list)

T = str((time.time() - start_time_0)/60)
minutes = T[:T.index(".")]
seconds = str(int(T[T.index(".")+1:])*60)[:2]
Time =  minutes+"m"+seconds +"s."
print(f"done,{len(main_df)} successfully scraped in {Time}")   

consulting  agents in houston-tx
done,250 successfully scraped in 2m66s.


In [4]:
main_df.head(5)

,Agent Name,Phone Number,Profile,Ratting,Employer
0,James Krueger,(713) 230-8620,https://www.zillow.com//profile/JamesKrueger/,5.0,Krueger Real Estate
1,Trinh Nguyen,(832) 646-0329,https://www.zillow.com//profile/TheTrinhNguyen/,4.5,The Realty
2,Chris Boyles,(832) 653-9603,https://www.zillow.com//profile/Chris-Boyles/,5.0,CB Realty
3,Skyler Tran,(832) 754-0662,https://www.zillow.com//profile/sellinginsneak...,5.0,The Good Label Real Estate
4,Johnathan Tran,(832) 784-8516,https://www.zillow.com//profile/johnathan312/,5.0,The Good Label Real Estate


In [5]:
main_df.to_csv(f'zillow {agent_type} agents in {search_location}.csv', index = False)

In [6]:
main_df = pd.read_csv(f'zillow {agent_type} agents in {search_location}.csv')

> ### Secondary Scraper . 

In [7]:
start_time_1 = time.time()
additional_info =[]
failed_urls = []
for url in main_df['Profile']:
    # sleep(2)

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
               "referer": url}    
    
    """ scraping the internet"""      
    webpage = requests.get(url = url, headers=headers)
    path_soup = BeautifulSoup(webpage.content,"html.parser")
    path = etree.HTML(str(path_soup))
    soup = BeautifulSoup(webpage.text,"html.parser")
    
    
    specialties_E = (path.xpath('//*[@id="__next"]/div/div[3]/main/div[2]/section/dl/dd'))
    specialties = ""
    for S in specialties_E:
        if specialties_E.index(S)==0:
            specialties += S.text 
        else:
            specialties += ", " + S.text 

    title = (path.xpath('//*[@id="__next"]/div/div[3]/aside/div[3]/div/dl/div/dt/span'))
    info = (path.xpath('//*[@id="__next"]/div/div[3]/aside/div[3]/div/dl/div/dd/span'))

    if len(info) == 0 and len(title) == 0 :
        title = (path.xpath('//*[@id="__next"]/div/div[3]/aside/div[2]/div/dl/div/dt/span'))
        info = (path.xpath('//*[@id="__next"]/div/div[3]/aside/div[2]/div/dl/div/dd/span'))

    websites = "Not provided"
    address = "Not provided"
    secondary_phone = "Not provided"
    primary_license = "Not provided"
    other_license = "Not provided"

    for T, I in zip(title, info):
        
        if T.text=="Websites":
            websites_E = (I.xpath('./a'))
            websites =""
            bypass = len(websites_E)
            for W in websites_E:
                site = W.text + ": "
                page = (W.xpath("./@href"))[0]+"\n"
                websites += site + page
                 
        elif T.text == "Broker address": 
            address_E = (I.xpath('./text()'))
            address = ""
            for A in address_E:
                address += A
                
        elif T.text == "Broker phone": 
            secondary_phone = I.text
            
        elif T.text == "Real Estate Licenses":
            primary_license = I.text
        
        elif T.text == "Other Licenses": 
            other_license = I.text

    contacts = {'Websites': websites, 'Address': address, 
                'Secondary Phone':  secondary_phone,'Real Estate Licenses': primary_license,
                'Other Licenses': other_license } 
    
    if websites == "Not provided" and address == "Not provided" and secondary_phone == "Not provided" and primary_license == "Not provided" and  other_license == "Not provided":
        failed_urls.append(url)
    
    additional_info.append(contacts)
    
info_df = pd.DataFrame(additional_info)

T = str((time.time() - start_time_1)/60)
minutes = T[:T.index(".")]
seconds = str(int(T[T.index(".")+1:])*60)[:2]
Time =  minutes+"m"+seconds +"s."
print(f"done,{len(info_df)} profiles were successfully scraped in {Time}")        

done,250 profiles were successfully scraped in 12m27s.


In [8]:
info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Websites              250 non-null    object
 1   Address               250 non-null    object
 2   Secondary Phone       250 non-null    object
 3   Real Estate Licenses  250 non-null    object
 4   Other Licenses        250 non-null    object
dtypes: object(5)
memory usage: 9.9+ KB


In [9]:
info_df.head(5)

,Websites,Address,Secondary Phone,Real Estate Licenses,Other Licenses
0,Website: http://www.krealestate.com\nBlog: htt...,"Krueger Real Estate3320 JacksonHouston, TX 77004",(346) 980-4783,573025 (TX),573025 (Real Estate Sales)
1,Website: https://www.har.com/trinh\nFacebook: ...,"The Realty9750 Bellaire Blvd, Suite 200Houston...",(832) 834-5872,Not provided,617251 (Real Estate Sales)
2,Website: https://cbrealtyteam.com\nFacebook: h...,"CB Realty2067 Jacquelyn DriveHouston, TX 77055",(832) 581-3661,595263 (TX),9003305 (Broker Owner)
3,Website: http://www.thegoodlabelre.com\nBlog: ...,The Good Label Real Estate24523 Foxberry Glen ...,Not provided,549761 (TX),0549761 (TX)
4,Not provided,The Good Label Real Estate1123 Bonner St. Hous...,Not provided,Not provided,Not provided


In [10]:
agents_df = main_df.join(info_df)

In [11]:
agents_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Agent Name            250 non-null    object 
 1   Phone Number          250 non-null    object 
 2   Profile               250 non-null    object 
 3   Ratting               250 non-null    float64
 4   Employer              250 non-null    object 
 5   Websites              250 non-null    object 
 6   Address               250 non-null    object 
 7   Secondary Phone       250 non-null    object 
 8   Real Estate Licenses  250 non-null    object 
 9   Other Licenses        250 non-null    object 
dtypes: float64(1), object(9)
memory usage: 19.7+ KB


In [12]:
agents_df.tail()

,Agent Name,Phone Number,Profile,Ratting,Employer,Websites,Address,Secondary Phone,Real Estate Licenses,Other Licenses
245,Chris Wooldridge,(832) 623-4268,https://www.zillow.com//profile/chris-properti...,5.0,CHRIS PROPERTIES,Website: https://www.chris-properties.com/\nBl...,"CHRIS PROPERTIES11111 Katy Freeway, Suite 910H...",(832) 623-4268,693037 (TX),0693037 (Texas)
246,Christine Fletcher,(281) 348-6514,https://www.zillow.com//profile/christinethere...,5.0,JLA Realty,Facebook: https://www.facebook.com/christine.f...,"JLA Realty18700 W Lake Houston Pkwy, B103Humbl...",Not provided,Not provided,Not provided
247,Christopher Newton,(281) 770-7932,https://www.zillow.com//profile/htowncribs/,5.0,O'Hara & Company Real Estate,Website: http://www.chrisnewtonrealtor.com\nFa...,O'Hara & Company Real Estate15855 Mueschke Rd....,Not provided,642770 (TX),642770 (Texas Real Estate License)
248,Christopher Raines,(346) 277-0283,https://www.zillow.com//profile/ChrisRaines3011/,5.0,Texas Realty,Facebook: https://www.facebook.com/ChrisLRaine...,Texas Realty12450 Bissonette Street Suite #122...,Not provided,737891 (TX),Not provided
249,Chuck Miller,(713) 906-6795,https://www.zillow.com//profile/Chuck-Miller4/,4.5,"Fanticular Real Estate, LLC",Website: http://www.Fanticular.com\nFacebook: ...,"Fanticular Real Estate, LLC2418 Richton StHous...",Not provided,Not provided,653675 (Real Estate Sales)


In [13]:
agents_df.to_csv(f'zillow {agent_type} agents in {search_location}.csv', index = False)